In [71]:
import pandas as pd 
import numpy as np 
import scipy.stats as stats
import matplotlib.pyplot as plt 
import folium
def clean_country_region (country_col):
    '''
    country_col is a string value
    returns
    clean_lst with extra spaces, asterisk, and date range e.g. (2008-2012) removed
    preserves country and region and demographic breakdown
    e.g. country_region should be e.g. "Ecuador, Quito"
    '''
    str1 = country_col.strip().replace('*','')
    str2 = str1.split('(')[0].strip()

    #print(str2)
    return str2

def std_country_region (country_region_col):
    '''
    country_col is a string value, standardize format
    ARGS:
        country_col String to transform
    RETURN:
        ISO recognized country name 
    '''
    dict = { 
        'Australian Capital Territory' : 'Australia, Australian Capital Territory',
        'Greater Poland':'Poland',
        'Iran':'Iran, Islamic Replublic of',
        'Republic of Korea':'Korea (South)',                  
        'South Australia':'Australia, South Australia',
        'The Netherlands':'Netherlands',
        'UK':'United Kingdom',
        'USA':'United States of America',
        'Western Australia':'Australia, Western Australia'
    }
    if (dict.get(country_region_col) == None):
        return country_region_col
    else:
        return dict.get(country_region_col)

def get_country_star(country_col):
    '''
    country_col is a Series/column which we'll treat as an iterable
    returns
    has star (bool)
    
    '''
    
    #print(country_col)
    idx = country_col.find('*')
    if idx == -1:
        return False
    else:
        return True

def get_country(country_col):
    '''
    Expects string with lots of extras
    returns only the country name
    '''
    str1 = country_col.strip() 
    str2 = str1.split(',')[0].replace('*','')
    str3 = str2.split('(')[0].strip()
    str4 = str3.split(':')[0]
    str5 = str4.split(';')[0]
    #print(str2)
    return str5


def set_national(country_col):
    # Returns true if the country long version let's you know it's national by not having : or ;
    if (country_col.find(':') == -1 and country_col.find(';') == -1 and country_col.find(',') == -1):
        return 1
    else:
        return 0

def set_is_subset(national_exists, is_national):
    # expects two integer values, 1 or 0 for T/F
    # Returns 0 if the national_exists is 1 and is_national is 0, else 1
    if (national_exists == 0 and is_national == 0):
        return 1
    else:   
        return 0


def munge_registry(df_registry):        

    df_registry['country_region'] = list(map(clean_country_region, df_registry['country_long']))
    # one off fix for Australia to make it roll up as a single country
    df_registry['country_region'] = list(map(std_country_region, df_registry['country_region']))

    df_registry['star'] = list(map(get_country_star, df_registry['country_long']))
    # change this to get country from cleansed country_region 
    df_registry['country_name'] = list(map(get_country, df_registry['country_region'])) 

    df_registry['is_national'] = list(map(set_national, df_registry['country_region']))
    # Some countries have cancer data at the national level and a sub-region by registry.  Others have regions that need to be summed.
    # Detect national registry by format of country name not containing ;:, then exclude all non national subsets for that country
    df_reg_group2 =  df_registry.groupby('country_name')['is_national'].sum().reset_index()
    #print('columns are {}'.format(df_registry.columns))

    # Zombie column wants to be here from last run, what the heck. 
    cols = ['is_national_exists', 'is_subset']
    for col in cols:
        if col in df_registry.columns:
            df_registry = df_registry.drop(columns=col, axis=1)
    df_registry.reset_index()        

    df_registry2 = df_registry.merge(df_reg_group2, left_on=['country_name'], right_on=['country_name'], suffixes=['', '_exists'])
    #print('columns are {}'.format(df_registry2.columns))
    # Trouble here, we seem to have to cols w/ same name, WTF, doesn't happen in Lab
    df_registry2['is_subset'] = list(map (set_is_subset, df_registry2['is_national_exists'], df_registry2['is_national']) )

    # use mask to eliminate redundant registries.  count goes from 464 to 273. 
    df_reduced = df_registry2[(df_registry2['is_national'] == 1 ) | (df_registry2['is_subset'] == 1)] 
    #df_reduced = df_registry2[(df_registry2['is_national'] == 1 ) | (df_registry2['is_national_exists'] == 0)] 
    df_reduced.set_index('REGISTRY')
    return df_reduced


def munge_meats(df_meat, df_milk, df_egg, year=2008):
    '''
    Accepts three data frames with annual meat, milk, and egg consumption data, cleans and reduceds them
    returns dataframe with joined, filtered, and summed data on kg per capital animal product consumption
    '''

    columns = (['country_name', 'country_code', 'year', 'bovine', 'poultry', 'pig', 'goat', 'other' ])
    df_meat.columns = columns
    df_meat2 = df_meat.fillna(0)
    df_meat2['meat'] = df_meat2['bovine'] + df_meat2['poultry']+ df_meat2['pig']+ df_meat2['goat']+ df_meat2['other']

    columns = (['country_name', 'country_code', 'year', 'milk'])
    df_milk.columns = columns
    df_milk2 = df_milk.fillna(0)

    columns = (['country_name', 'country_code', 'year', 'egg'])
    df_egg.columns = columns
    df_egg2 = df_egg.fillna(0)

    # filter year 
    df_meat3 = df_meat2[df_meat2['year'] == year]  #209 records expected
    df_milk3 = df_milk2[df_milk2['year'] == year]  #209
    df_egg3 = df_egg2[df_egg2['year'] == year]


    df_animal1 = df_meat3.merge(df_milk3, left_on='country_name', right_on = 'country_name', suffixes=('', '_milk'))
    df_animal2 = df_animal1.merge(df_egg3, left_on='country_name', right_on = 'country_name', suffixes=('', '_egg'))

    # sum
    df_animal2['animal_product_kg_cap_yr'] = df_animal2['meat'] + df_animal2['milk'] + df_animal2['egg'] 

    # select only columns we care about
    df_animal3 = df_animal2.loc[:,('country_name', 'country_code', 'year', 'animal_product_kg_cap_yr')]
    return df_animal3

def turn_cancer_to_per_capita(df_cases, df_pop, df_reg):
    # returns reduced dataframe for registries we want, summed and merged with population data
    # Note: I can decide to accept df's as args (functional) or use pseudo-oop here w df's part of the state
    
    df_cases.drop(['N_unk','N85', 'N80_84', 'N75_79','N70_74','N65_69','N60_64','N55_59','N50_54','N0_4','N5_9','N10_14','N15_19'], axis=1, inplace=True)
    df_cases['N20_49'] = df_cases['N20_24'] + df_cases['N25_29'] + df_cases['N30_34'] + df_cases['N35_39'] + df_cases['N40_44'] + df_cases['N45_49']
    df_case_sum = df_cases.groupby(['REGISTRY']).sum()['N20_49'].reset_index()

    df_pop.drop(['P_unk','P85', 'P80_84', 'P75_79','P70_74','P65_69','P60_64','P55_59','P50_54','P0_4','P5_9','P10_14','P15_19'], axis=1, inplace=True)
    df_pop['P20_49'] = df_pop['P20_24'] + df_pop['P25_29'] + df_pop['P30_34'] + df_pop['P35_39'] + df_pop['P40_44'] + df_pop['P45_49']
    df_pop_sum = df_pop.groupby(['REGISTRY']).sum()['P20_49'].reset_index()

    df_case_v_pop = df_case_sum.merge(df_pop_sum, left_on='REGISTRY', right_on='REGISTRY', suffixes=('', '_pop'))

    df_case_v_pop_reg = df_case_v_pop.merge(df_reg, left_on='REGISTRY', right_on='REGISTRY', suffixes=('', '_lu'))[['REGISTRY', 'N20_49', 'P20_49', 'country_name']]
    df_case_v_pop_country = df_case_v_pop_reg.groupby('country_name').agg({'N20_49':sum, 'P20_49':sum}).reset_index()
    df_case_v_pop_country.sort_values('country_name')

    df_case_v_pop_country['Incidence Per Age Capita'] = df_case_v_pop_country['N20_49'] / df_case_v_pop_country['P20_49'] 

    return df_case_v_pop_country

def cancer_pc_by_country (df_cancer, df_all_countries):
     # returns cancer per capita joined with the country table, reduced to our set of countries


    df_cancer_pc_by_country = df_cancer.merge(df_all_countries, how='left', left_on="country_name", right_on='Name', suffixes = ('', 'iso'))

    return df_cancer_pc_by_country

def get_cancer_isos(df_cancer, df_iso):

    df_results = df_cancer.merge(df_iso, left_on="country_name", right_on = "Name", suffixes = ('', '_iso'))
    return df_results
    pass

def get_animal_isos(df_animal, df_iso):

    df_results = df_animal.merge(df_iso, left_on='country_code', right_on = 'Alpha3')  
    return df_results



In [72]:

def read_cancer_data(filename):
    # Returns data frame with cancer data
    df_cases = pd.read_csv(filename)  # ('../data/CI5-XI/cases.csv')
    df_cases.set_index('REGISTRY')
    return df_cases

def read_population_data(filename):
    # Returns data frame with cancer data
    df_pop = pd.read_csv(filename)  # (../data/CI5-XI/pop.csv')
    return df_pop

def read_cancer_registry(filename):
        # Returns data frame with cancer data
    df_registry = pd.read_csv(filename, sep = "\t", header=0, encoding='iso-8859-1') # '../data/CI5-XI/cancer_summary.txt'
    # cleansing
    df_registry.columns = (['REGISTRY', 'country_long'])
    df_reg = munge_registry(df_registry)
    return df_reg

def dump_df(filename, df):
    df.to_csv(filename)    

def reload_df_utf8(filename):
    df = pd.read_csv(filename, encoding='utf-8')
    return df    

def combine_data(df_cancer_data, df_animal_cons):
    df_combo = df_cancer_data.merge(df_animal_cons, on="Alpha3", suffixes=('', '_animal'))
    df_combo2 = df_combo.loc[:,('country_name', 'Alpha3', 'N20_49', 'P20_49', 'Incidence Per Age Capita', 'year', 'animal_product_kg_cap_yr')]

    return df_combo2


if __name__ == '__main__':
    '''
    todo: We'll use this to accept filenames as parameteris
    ap = argparse.ArgumentParser()
    ap.add_argument("-f", "--file", required=True, help="text file containing words")
    ap.add_argument("-m", "--mode", required=True, help="""mode determining
                    output. single: computation times based on one random
                    selection from word list, average: computation times based
                    on average of 10 random selections from word list""")
    ap.add_argument("-t", "--fit", required=True, help="""True or False: plot
                    polynomial fit on plots""")
    args = vars(ap.parse_args())

    if args['mode'] == 'single': ...
    '''
    # Cancer and population data from WHO
    df_cases = read_cancer_data('../data/CI5-XI/cases.csv')
    df_pop = read_population_data('../data/CI5-XI/pop.csv')

    # Lookup tables
    df_registry = read_cancer_registry('../data/CI5-XI/registry.txt')
    df_all_countries = pd.read_csv('../data/country_iso.csv')

    # Meat and animal product consumption data from FAO via ___
    df_meat = pd.read_csv('../data/FAO/per-capita-meat-consumption-by-type-kilograms-per-year.csv')
    df_milk = pd.read_csv('../data/FAO/per-capita-milk-consumption.csv')
    df_egg = pd.read_csv('../data/FAO/per-capita-egg-consumption-kilograms-per-year.csv')

    print('Number of rows in cases: {}'.format(len(df_cases)))
    print('Number of rows in pop: {}'.format(len(df_pop)))
    print('Number of rows in registry: {}'.format(len(df_registry)))

    print('Number of rows in raw meat, milk, egg data (mult year): {}, {}, {}'.format(len(df_meat), len(df_milk), len(df_egg)))
    print('Number of rows in country iso : {}'.format(len(df_all_countries)))

    # Munging the registry data, already done.
    # df_registry2 = munge_registry(df_registry)

    # Removed because it's not working correctly.  Using fixed data file now.
    dump_df('../data/clean_registry.csv', df_registry)
    df_reg = reload_df_utf8('../data/clean_registry.csv')
    print('Reloaded reg UTF-8 Number of rows in registry: {}'.format(len(df_reg)))

    df_cancer_data = turn_cancer_to_per_capita(df_cases, df_pop, df_reg)
    df_cancer_by_iso = get_cancer_isos(df_cancer_data, df_all_countries)




Number of rows in cases: 58590
Number of rows in pop: 930
Number of rows in registry: 440
Number of rows in raw meat, milk, egg data (mult year): 11028, 11028, 11028
Number of rows in country iso : 247
Reloaded reg UTF-8 Number of rows in registry: 440


In [73]:
df_registry[df_registry['country_name']== 'Costa Rica' ]
df_cr = df_cases[(df_cases['REGISTRY']== 218800099) & (df_cases['CANCER'] == 62)]
df_cr.groupby('REGISTRY').sum()


,SEX,CANCER,TOTAL,N20_24,N25_29,N30_34,N35_39,N40_44,N45_49,N20_49
REGISTRY,,,,,,,,,,
218800099,3,124,39259,496,757,1003,1340,2028,2784,8408


In [74]:
df_registry[df_registry['country_region']== 'United States of America' ]
df_us = df_cases[(df_cases['REGISTRY']== 384000099) & (df_cases['CANCER'] == 62)]
df_us.groupby('REGISTRY').sum()
#df_us.sort_values(['TOTAL'])


,SEX,CANCER,TOTAL,N20_24,N25_29,N30_34,N35_39,N40_44,N45_49,N20_49
REGISTRY,,,,,,,,,,
384000099,3,124,7555489,38040,61245,90401,136801,230550,400298,957335


In [75]:
df_pop.head()


,REGISTRY,SEX,AGE_GROUPS,P20_24,P25_29,P30_34,P35_39,P40_44,P45_49,P20_49
0,101200199,1,16,367163,341413,280809,220833,176887,151974,1539079
1,101200199,2,16,350207,327421,271908,214591,172801,150313,1487241
2,101200399,1,16,338002,282582,218117,174979,154664,136651,1304995
3,101200399,2,16,328725,271638,217611,180110,160956,142956,1301996
4,140400299,1,16,1055445,1172980,913115,672295,445545,329505,4588885


In [76]:
df_pop = read_population_data('../data/CI5-XI/pop.csv')
df_pop.head()

,REGISTRY,SEX,AGE_GROUPS,P0_4,P5_9,P10_14,P15_19,P20_24,P25_29,P30_34,...,P45_49,P50_54,P55_59,P60_64,P65_69,P70_74,P75_79,P80_84,P85,P_unk
0,101200199,1,16,279377,259962,280726,338057,367163,341413,280809,...,151974,123872,98670,59989,41303,38654,58885,0,0,0
1,101200199,2,16,263105,247079,267080,321907,350207,327421,271908,...,150313,121188,93566,62372,49045,45806,65590,0,0,0
2,101200399,1,16,300292,260233,305477,347657,338002,282582,218117,...,136651,110090,88675,53498,48356,39705,58531,0,0,0
3,101200399,2,16,284900,249660,292453,334301,328725,271638,217611,...,142956,111205,83725,54288,49574,41371,57232,0,0,0
4,140400299,1,16,996905,759500,599755,578860,1055445,1172980,913115,...,329505,208410,121520,75305,36790,22310,28860,0,0,0


In [77]:
df_pop['Total_Pop'] = df_pop['P0_4'] + df_pop['P5_9'] + df_pop['P10_14'] + df_pop['P15_19'] + df_pop['P20_24'] + df_pop['P25_29'] + df_pop['P30_34'] + df_pop['P35_39'] + df_pop['P40_44'] + df_pop['P45_49'] + df_pop['P50_54'] + df_pop['P55_59'] + df_pop['P60_64'] + df_pop['P65_69'] + df_pop['P70_74'] +  df_pop['P75_79'] +  df_pop['P80_84'] +  df_pop['P85'] +  df_pop['P_unk']  

In [78]:
df_pop[df_pop['REGISTRY']== 384000099].groupby('REGISTRY').sum()

,SEX,AGE_GROUPS,P0_4,P5_9,P10_14,P15_19,P20_24,P25_29,P30_34,P35_39,...,P50_54,P55_59,P60_64,P65_69,P70_74,P75_79,P80_84,P85,P_unk,Total_Pop
REGISTRY,,,,,,,,,,,,,,,,,,,,,
384000099,3,36,100811376,101255944,103415760,109377288,109010212,105807760,100441244,100486412,...,111145784,98926228,84082340,63074728,47006172,36835345,28778377,27697926,0,1545634484


In [79]:
def annualize_pop_data(df_pop):
    # We have 5 years data for each country, divide each stat by 5

    # for each column (this can be written much more elegantly)
    df_pop['Total_Pop'] = df_pop['Total_Pop'] / 5
    df_pop['P20_24'] = df_pop['P20_24'] / 5
    df_pop['P25_29'] = df_pop['P25_29'] / 5
    df_pop['P30_34'] = df_pop['P30_34'] / 5
    df_pop['P35_39'] = df_pop['P35_39'] / 5
    df_pop['P40_44'] = df_pop['P40_44'] / 5
    df_pop['P45_49'] = df_pop['P45_49'] / 5
    # The rest get deleted anyway

    return df_pop

df_pop2 = annualize_pop_data(df_pop)

 

In [81]:
df_pop2[df_pop2['REGISTRY']== 218800099].groupby('REGISTRY').sum()

,SEX,AGE_GROUPS,P0_4,P5_9,P10_14,P15_19,P20_24,P25_29,P30_34,P35_39,...,P50_54,P55_59,P60_64,P65_69,P70_74,P75_79,P80_84,P85,P_unk,Total_Pop
REGISTRY,,,,,,,,,,,,,,,,,,,,,
218800099,3,36,1412056,1497040,1624705,1714196,350105.0,321507.4,283398.6,248606.8,...,978546,745159,544585,395489,301619,219663,147390,115210,0,3627910.4


In [89]:
df_meat[(df_meat['Code']=='USA') & (df_meat['Year'] == 2008)]

,Entity,Code,Year,"Bovine meat food supply quantity (kg/capita/yr) (FAO, 2020)","Poultry meat food supply quantity (kg/capita/yr) (FAO, 2020)","Pigmeat food supply quantity (kg/capita/yr) (FAO, 2020)","Mutton & Goat meat food supply quantity (kg/capita/yr) (FAO, 2020)","Meat, Other, Food supply quantity (kg/capita/yr) (FAO, 2020)"
10403,United States,USA,2008,40.58,51.61,29.08,0.5,0.84


In [90]:
df_egg[(df_egg['Code']=='USA') & (df_egg['Year'] == 2008)]

,Entity,Code,Year,"Eggs - Food supply quantity (kg/capita/yr) (FAO, 2020)"
10403,United States,USA,2008,13.98


In [91]:
df_milk[(df_milk['Code']=='USA') & (df_milk['Year'] == 2008)]

,Entity,Code,Year,"Milk - Excluding Butter - Food supply quantity (kg/capita/yr) (FAO, 2020)"
10403,United States,USA,2008,253.39
